<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_MLP/blob/main/MLP_tf2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun, JANG
# limitsinx.tistory.com

import numpy as np
import pandas as pd
import tensorflow as tf
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
tf.random.set_seed(1)

#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/test_v.txt')
train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]
print(train_data)
print(test_data)

In [ ]:
#Normalization
def MinMaxScaler(data):
  denom = np.max(data,0) - np.min(data,0)
  nume = data-np.min(data,0)
  return nume/denom

def back_MinMax(data,max,min):
  diff = max-min
  back = data * diff + min
  return back

In [ ]:
#Hyper parameters
seqLength = 0
dataDim = df_test.shape[1]
outputDim = 1
hiddenDim = 4
lr = 0.002
iterations = 10
batch_size = 16

trainSize = int(len(train_data))
trainSet = train_data[0:trainSize]
testSize = int(len(test_data))
testSet = test_data[0:testSize]
testSet_for_result = testSet

trainSet_max = np.max(trainSet)
trainSet_min = np.min(trainSet)
testSet_max = np.max(testSet)
testSet_min = np.min(testSet)
trainSet_max = (trainSet_max[dataDim-1])
trainSet_min = (trainSet_min[dataDim-1])
testSet_max = (testSet_max[dataDim-1])
testSet_min = (testSet_min[dataDim-1])


trainSet = MinMaxScaler(trainSet)
testSet = MinMaxScaler(testSet)

print(trainSet.shape) # (5807,5)
print(testSet.shape) # (10882,5)

In [ ]:
# Data Preprocessing
input_dim = trainSet.shape[1]-1
trainX = trainSet.iloc[:,0:input_dim]
trainY = trainSet.iloc[:,-1]
testX = testSet.iloc[:,0:input_dim]
testY = testSet.iloc[:,-1]

In [ ]:
# Multi Layer Perceptron(MLP) Modeling
# Activation function "tanh" : Glorot normal initializer
# Activation function "reLU" : He Initiailizer
from tensorflow import keras
from tensorflow.keras import layers

initializer = tf.keras.initializers.GlorotNormal()
input_dim = trainSet.shape[1]-1
model = keras.Sequential()
model.add(layers.Dense(units = hiddenDim, activation='linear',input_dim = input_dim, kernel_initializer = initializer))
model.add(layers.Dense(units = hiddenDim, activation = 'linear', kernel_initializer = initializer))
model.add(layers.Dense(units = hiddenDim, activation = 'linear', kernel_initializer = initializer))
model.add(layers.Dense(units = hiddenDim, activation = 'linear', kernel_initializer = initializer))
model.add(layers.Dense(outputDim, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model.compile(optimizer=adam, loss='mse')
model.summary()


In [ ]:
# Training
hist = model.fit(trainX, trainY, epochs = iterations, batch_size = batch_size)

In [ ]:
# Evaluate Testset
res = model.evaluate(testX, testY, batch_size = batch_size)
print(res)
#res_batch_size1 = model.evaluate(testX, testY, batch_size = 1) #Very Slow


In [ ]:
# Estimating
xhat = testX
yhat = model.predict(xhat)
testY = testY.to_numpy()
print("test Y : ", testY, testY.shape,type(testY))
print("yhat : ",yhat, yhat.shape,type(yhat))

print("RMSE : {}".format(np.sqrt(np.average((yhat - testY)**2))))

In [ ]:
plt.figure()
plt.plot(estimated[:len(estimated)],label = "Estimated by MLP")
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})